<a href="https://colab.research.google.com/github/Imsachin010/Delhivery_Address_Classifier/blob/main/Delhivery_adress_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Delhivery/Assignment - Address Tag/assignment_data_res_com.csv')
df.head(5)

,id,masked_address,label
0,2,"JAI BALAJI DISTRIBUTORS, NEW MARKET SHOP NO.**...",commercial
1,423,SUNDRAM FASTENERS LIMITED B ** SIPCOT INDUSTRI...,commercial
2,481,"Bakul-*, Shardashish Township, Torrent Power L...",commercial
3,641,"Sonkar Bhawan, Near Ashok Kirana Store, Infron...",commercial
4,4395,"jambli \n maida factory near jambli , jambli,...",commercial


In [21]:
# Assuming 'label' column contains 'commercial' or 'residential'
# Replace 'label' with your actual label column
X = df['masked_address']
y = df['label']


In [22]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature extraction using CountVectorizer
vectorizer = CountVectorizer(stop_words=stopwords.words('english'))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [23]:
# Remove NaN values
df = df.dropna()
df.head(5)

,id,masked_address,label
0,2,"JAI BALAJI DISTRIBUTORS, NEW MARKET SHOP NO.**...",commercial
1,423,SUNDRAM FASTENERS LIMITED B ** SIPCOT INDUSTRI...,commercial
2,481,"Bakul-*, Shardashish Township, Torrent Power L...",commercial
3,641,"Sonkar Bhawan, Near Ashok Kirana Store, Infron...",commercial
4,4395,"jambli \n maida factory near jambli , jambli,...",commercial


In [24]:
# Remove NaN values
df = df.dropna()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['masked_address'], df['label'], test_size=0.2, random_state=550)

# Feature extraction using CountVectorizer
vectorizer = CountVectorizer(stop_words=stopwords.words('english'))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a logistic regression model
classifier = LogisticRegression()
classifier.fit(X_train_vectorized, y_train)

# Predictions on the test set
y_pred = classifier.predict(X_test_vectorized)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

  commercial       0.00      0.00      0.00         3
 residential       0.40      1.00      0.57         2

    accuracy                           0.40         5
   macro avg       0.20      0.50      0.29         5
weighted avg       0.16      0.40      0.23         5



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# For example, assuming you have a new address 'new_address':
new_address = ["OFFICE NO-***, *TH FLOOR, BLOCK-EP /GP, SEC-V, SALT LAKE CITY, KOLKATA,****  , kolkata, west bengal, 700091"]
new_address_vectorized = vectorizer.transform(new_address)
prediction = classifier.predict(new_address_vectorized)
print(f"Prediction for the new address: {prediction}")

Prediction for the new address: ['commercial']


In [26]:
# For example, assuming you have a new address 'new_address':
new_address = ["Village kandi p.o.khaniyara.teh Dharamshala  Khanyara road  , Kangra District, Himachal Pradesh, 176218"]
new_address_vectorized = vectorizer.transform(new_address)
prediction = classifier.predict(new_address_vectorized)
print(f"Prediction for the new address: {prediction}")

Prediction for the new address: ['residential']


# **Hypertuning**

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression


In [28]:
# We have preprocessed data and labels
X = df['masked_address']
y = df['label']

In [29]:
# Create a pipeline with CountVectorizer and LogisticRegression
pipeline = make_pipeline(
    CountVectorizer(stop_words=stopwords.words('english')),
    LogisticRegression()
)

# Define hyperparameter grid for tuning
param_grid = {
    'countvectorizer__ngram_range': [(1, 1), (1, 2)],  # Adjust the ngram_range
    'logisticregression__C': [0.1, 1, 10],  # Adjust the regularization parameter
}

In [30]:
# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)


Best Hyperparameters: {'countvectorizer__ngram_range': (1, 2), 'logisticregression__C': 0.1}


In [31]:
# Use the best model for prediction
best_model = grid_search.best_estimator_

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the best model on the training data
best_model.fit(X_train, y_train)

# Predictions on the test set
y_pred = best_model.predict(X_test)

In [32]:
# Additional evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='commercial', average='binary')
recall = recall_score(y_test, y_pred, pos_label='commercial', average='binary')
f1 = f1_score(y_test, y_pred, pos_label='commercial', average='binary')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.4
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
Confusion Matrix:
 [[0 3]
 [0 2]]


# **Validation**

In [34]:
  # For example, assuming you have a new address 'new_address':
new_address = ["Village kandi p.o.khaniyara.teh Dharamshala  Khanyara road  , Kangra District, Himachal Pradesh, 176218"]
new_address_vectorized = vectorizer.transform(new_address)
prediction = classifier.predict(new_address_vectorized)
print(f"Prediction for the new address: {prediction}")

Prediction for the new address: ['residential']


# **Using NLP**

In [35]:
!pip install tensorflow
!pip install scikit-learn

In [36]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [37]:
# # Assuming you have X (text data) and y (labels) defined
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [38]:
# Pad sequences to ensure equal length
X_train_padded = pad_sequences(X_train_sequences)
X_test_padded = pad_sequences(X_test_sequences, maxlen=X_train_padded.shape[1])

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [39]:
# Define the LSTM model
embedding_dim = 50  # Adjust as needed
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=X_train_padded.shape[1]))
model.add(LSTM(units=1000))
model.add(Dense(units=1, activation='sigmoid'))  # Binary classification, so using sigmoid activation

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_encoded, epochs=150, batch_size=32, validation_split=0.2)

Epoch 1/150
1/1 [==============================] - 4s 4s/step - loss: 0.6934 - accuracy: 0.3333 - val_loss: 0.6876 - val_accuracy: 0.5000
Epoch 2/150
1/1 [==============================] - 0s 45ms/step - loss: 0.6843 - accuracy: 0.9167 - val_loss: 0.6757 - val_accuracy: 0.7500
Epoch 3/150
1/1 [==============================] - 0s 43ms/step - loss: 0.6696 - accuracy: 1.0000 - val_loss: 0.6440 - val_accuracy: 0.5000
Epoch 4/150
1/1 [==============================] - 0s 43ms/step - loss: 0.6395 - accuracy: 0.6667 - val_loss: 0.5952 - val_accuracy: 0.7500
Epoch 5/150
1/1 [==============================] - 0s 44ms/step - loss: 0.5707 - accuracy: 0.9167 - val_loss: 0.4668 - val_accuracy: 1.0000
Epoch 6/150
1/1 [==============================] - 0s 44ms/step - loss: 0.4485 - accuracy: 0.7500 - val_loss: 6.3733 - val_accuracy: 0.5000
Epoch 7/150
1/1 [==============================] - 0s 43ms/step - loss: 5.6000 - accuracy: 0.5000 - val_loss: 0.4620 - val_accuracy: 1.0000
Epoch 8/150
1/1 [=====

In [40]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f"Test Accuracy: {accuracy}")

# Make predictions
y_pred_prob = model.predict(X_test_padded)
y_pred = (y_pred_prob > 0.5).astype(int)

1/1 [==============================] - 0s 113ms/step - loss: 3.7352 - accuracy: 0.4000
Test Accuracy: 0.4000000059604645
1/1 [==============================] - 2s 2s/step


In [41]:
# Additional evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy:", accuracy_score(y_test_encoded, y_pred))
print("Precision:", precision_score(y_test_encoded, y_pred))
print("Recall:", recall_score(y_test_encoded, y_pred))
print("F1 Score:", f1_score(y_test_encoded, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test_encoded, y_pred))


Accuracy: 0.4
Precision: 0.4
Recall: 1.0
F1 Score: 0.5714285714285715
Confusion Matrix:
 [[0 3]
 [0 2]]


In [42]:
def preprocess_new_address(new_address):
    # Tokenize and pad the new address
    new_address_sequence = tokenizer.texts_to_sequences([new_address])
    new_address_padded = pad_sequences(new_address_sequence, maxlen=X_train_padded.shape[1])
    return new_address_padded

def predict_address_classification(new_address):
    # Preprocess the new address
    new_address_padded = preprocess_new_address(new_address)

    # Make the prediction
    prediction_prob = model.predict(new_address_padded)
    prediction = (prediction_prob > 0.5).astype(int)

    # Decode the prediction
    predicted_class = label_encoder.inverse_transform(prediction)[0]

    return predicted_class

# **Validation**

In [43]:
# Example: Predicting the classification for a new address
new_address = "Village kandi p.o.khaniyara.teh Dharamshala  Khanyara road  , Kangra District, Himachal Pradesh, 176218"
predicted_class = predict_address_classification(new_address)

print(f"The predicted classification for the address is: {predicted_class}")

1/1 [==============================] - 0s 203ms/step
The predicted classification for the address is: residential


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [46]:
# Example: Predicting the classification for a new address
new_address = "GOPALPUR WAREHOUSING CMPLX BASAI GOPALPUR , KHASRA NO.** QILA NO.** , GURGAON  , Gurgaon, Haryana, 122001"
predicted_class = predict_address_classification(new_address)

print(f"The predicted classification for the address is: {predicted_class}")

1/1 [==============================] - 0s 32ms/step
The predicted classification for the address is: commercial


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
